<a href="https://colab.research.google.com/github/ehabosaleh/Face-expressions-detector/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import the DataSet
!git clone https://github.com/misbah4064/facial_expressions.git
#   تحميل قاعدة بيانات تمثل نماذج عن التعابير الوجهية

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14243, done.
remote: Total 14243 (delta 0), reused 0 (delta 0), pack-reused 14243
Receiving objects: 100% (14243/14243), 240.06 MiB | 42.95 MiB/s, done.
Resolving deltas: 100% (232/232), done.
Checking out files: 100% (14004/14004), done.


In [ ]:
!ls

anger.txt   facial_expressions			 neutral.txt  surprise.txt
data	    haarcascade_frontalface_default.xml  python       test
data_set    happy.txt				 R
dwayne.jpg  images				 README.md
elon.jpg    LICENSE				 sad.txt


In [ ]:
!pip3 install opencv-python

In [ ]:
#Creating All Necessary Directories
%cd facial_expressions/
%mkdir --parents  data_set/{anger,happy,neutral,sad,surprise}
#نشاء مجلدات يحتوي كل واحد على تعبير وجه محدد

/content/facial_expressions


In [ ]:
#Extracting Images With Expressions
import  cv2
with  open('happy.txt','r')  as  f:
  img = [line.strip() for line in f]
for image in img:
  loadedImage = cv2.imread("images/"+image)
  cv2.imwrite("data_set/happy/"+image,loadedImage)
print("done writing")
#   باللاستفادة من المكتبةcv2 قراءة الصور التي تمثل تعبير الفرح 

done writing


In [ ]:
%mkdir dataset

In [ ]:
#First we create Data Set of Faces
import cv2

with open('happy.txt','r') as f:
  images = [line.strip() for line in f]

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# For each Emotion, enter one numeric face id
face_id = input('\n Enter Emotion id end press <return> ==>  ')

count = 0

for image in images:
  img = cv2.imread("data_set/happy/"+image)
  gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = face_detector.detectMultiScale(gray, 1.3, 5)

  for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])

print("\n Done creating face data")
# اكتشاف نماذج الوجوه من كل صورة في قاعدة البيانات و حفظها


 Enter Emotion id end press <return> ==>  1

 Done creating face data


In [ ]:
%mkdir trainer

mkdir: cannot create directory ‘trainer’: File exists


In [ ]:
#Second we train all images
import cv2
import numpy as np
from PIL import Image
import os

# Path for face image database
path = 'dataset'

recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    faceSamples=[]
    ids = []

    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        img_numpy = np.array(PIL_img,'uint8')

        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)

        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return  faceSamples,ids

print ("\n [INFO] Training faces....")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml')

# Print the numer of Emotions trained and end program
print("\n [INFO] {0} Emotions trained. Exiting Program".format(len(np.unique(ids))))
#تدريب المصنف باستخدام النماذج التي تم استخلاصها من الكود السابق


 [INFO] Training faces....

 [INFO] 1 Emotions trained. Exiting Program


In [ ]:
# Now we do the testing or Recognition
import cv2
import numpy as np
import os

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

#iniciate id counter
id = 0

# Emotions related to ids: example ==> Anger: id=0,  etc
names = ['Anger', 'Happy', 'None', 'None', 'None', 'None']

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)

# ret, img =cam.read()
img = cv2.imread("elon.jpg")
# img = cv2.flip(img, -1) # Flip vertically

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale(gray,scaleFactor = 1.2, minNeighbors = 5, minSize = (int(minW), int(minH)),)

for(x,y,w,h) in faces:

    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

    id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

    # Check if confidence is less them 100 ==> "0" is perfect match 
    if (confidence < 100):
        id = names[id]
        confidence = "  {0}%".format(round(100 - confidence))
    else:
        id = "unknown"
        confidence = "  {0}%".format(round(100 - confidence))
    
    cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
    cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  

cv2.imwrite("elonTest.jpg",img) 

print("\n [INFO] Done detecting and Image is saved")
cam.release()
cv2.destroyAllWindows()
#اختبار المصنف بعد أن قمنا ببناءه في الكود السابق

SyntaxError: ignored

In [ ]:
#Also we can display the outpot
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

image = cv2.imread("elonTest.jpg")
height, width = image.shape[:2]
resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

fig = plt.gcf()
fig.set_size_inches(18, 10)
plt.axis("off")
plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
plt.show()
#اظهار نموذج كمثال من اختبار المصنف

SyntaxError: ignored